In [1]:
#Import Section
import re
import pandas as pd
import requests
import numpy as np
import time
import datetime
import string
import gensim
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora

In [6]:
#Defination of Functions

#to get a list of link from the search query on youtube
def load_page(scrape_url, search_hardcode):
    mozhdr = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-GB; rv:1.9.0.3) Gecko/2008092417 Firefox/3.0.3'}
    requests.get("https://www.youtube.com", headers = mozhdr)

    sb_url = scrape_url + search_hardcode + "/"
    print(sb_url)
    sb_get = requests.get(sb_url, headers = mozhdr)
    sb_get.content
    
    soupeddata = BeautifulSoup(sb_get.content, "html.parser")
    yt_links = soupeddata.find_all("a", class_ = "yt-uix-tile-link") # might need to change

    for x in yt_links:
        yt_href = x.get("href")
        yt_title = x.get("title")
        yt_final = scrape_url + yt_href
        print(yt_final)
    return soupeddata


#Extract title from the soupeddata
def get_title(soupeddata):
    print (soupeddata.title)
    heading = re.findall(r'>([\w+\s]+)',str(soupeddata.title))
    return heading


#Extract title and text data from the soupeddata
def get_text(soupeddata):
    for script in soupeddata(["script", "style"]):
        script.extract()
    text = soupeddata.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    #separating title and converting text_body into list to work on
    heading_test = text.splitlines()[0]
    text_body = text.splitlines()[1:]
    return text_body,heading_test



#Start Topic Modeling Process

#Clean Doc 
def clean(doc):
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    some_more = {'>>', '%' , '$', '/', '#', ':', ',', '.' }
    lemma = WordNetLemmatizer()
    normalized_stemmed = ""
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    some_more_free = ''.join(ch for ch in punc_free if ch not in some_more)
    
    for word in punc_free.split():
        y = lemma.lemmatize(word,"v") #lemmatize all words for verb
        y = lemma.lemmatize(y,"a")    #lemmatize all words for adjectives
        y = lemma.lemmatize(y,"n")    #lemmatize all words for nouns
                           
        normalized_stemmed = normalized_stemmed + " " + y
   
    return normalized_stemmed





# Preparing Document-Term Matrix for text modelling

def make_model(doc_clean,n_topics):
    # Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
    dictionary = corpora.Dictionary(doc_clean)
    print(dictionary)
    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    
    # Creating the object for LDA model using gensim library
    Lda = gensim.models.ldamodel.LdaModel    
    # Running and Trainign LDA model on the document term matrix.
    ldamodel = Lda(doc_term_matrix, num_topics=n_topics, id2word = dictionary, passes=50)
    return ldamodel

#RESULT
def pred_title(ldamodel, n_topics, n_words):
    headings_pred = ldamodel.print_topics(num_topics=n_topics, num_words=n_words)
    all_words = re.findall(r'"([\w]+)"',str(headings_pred))
    return all_words

def freq(all_words):
    frequency = {}
    for word in all_words:
        count = frequency.get(word,0)   
        frequency[word] = count + 1


    frequency_list = frequency.keys()

    f1 = {}
    for k,v in frequency.items():
        if(v>2):
            f1[k] = v

    for k,v in sorted(f1.items(), key=lambda p:p[1], reverse=True):
        print(k,v)
    


# Compare accuracies of predicted topics with the actual one

def comp_accu(head_test_split_norm, all_words, stop_words):
    accuracy_count =0

    for y in head_test_split_norm:
        for x in all_words:

            if (x == y):
                print(y, x)
                accuracy_count = accuracy_count+1
                break
    return accuracy_count

def head_count(doc):
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    some_more = {'>>', '%' , '$', '/', '#', ':', ',', '.' , '?', ':'}
    lemma = WordNetLemmatizer()
    normalized_stemmed = ""
    print(doc)
    punc_free = " ".join(ch for ch in doc.lower().split() if ch not in exclude)
    print(punc_free)
    some_more_free = " ".join(ch for ch in punc_free.split() if ch not in some_more)
    print(some_more_free)
    total_count = len(some_more_free.split())
    print(total_count)
    stop_free = " ".join([i for i in some_more_free.split() if i not in stop])
    print(stop_free)
    after_count =len(stop_free.split())
    print(after_count)
    for word in punc_free.split():
        y = lemma.lemmatize(word,"v") #lemmatize all words for verb
        y = lemma.lemmatize(y,"a")    #lemmatize all words for adjectives
        y = lemma.lemmatize(y,"n")    #lemmatize all words for nouns
                           
        normalized_stemmed = normalized_stemmed + " " + y
    temp = (total_count-after_count)
    return total_count, after_count, temp
    
                
def show_results(accuracy_count, n_topics, n_words, stop_words):
    print("\nAverage Accuracy in",n_topics,"topics =",(accuracy_count/(n_words-stop_words)*100),"%")

In [19]:
#MAIN

base="https://bloggerspassion.com/"
niche1 = "best-real-estate-affiliate-programs" 
niche2 = "how-to-make-money-online-in-india"
niche3 = "hostgator-hosting-plans"
niche4 = "best-fashion-blogs-in-india"
niche5 = "off-page-seo"

base2 = "https://www.labnol.org/"
niche6 = "internet/reliance-jio-phone-ads/31965/"
niche7 = "internet/gmail-attachment-reminder/31959/"
niche8 = "internet/101-useful-websites/18078/"
niche9 = "internet/create-twitter-screenshots/13842/"
niche10 = "internet/buying-udemy-courses/31851/"

  #number of stop words in title
number_of_topics_pred = 5 #number of titles to be predicted for maximum accuracy

soupeddata1 = load_page(base2, niche10)
blog_title = get_title(soupeddata1)
blog_body, heading_test1 = get_text(soupeddata1)
number_of_words_in_title, number_of_words_without_stopwords, number_of_stop_words = head_count(heading_test1)
cleaned_heading = clean(heading_test1)
cleaned_document = [clean(doc).split() for doc in blog_body]
lda_model = make_model(cleaned_document, number_of_topics_pred)
all_pred_heads = pred_title(lda_model, number_of_topics_pred, number_of_words_in_title)
freq(all_pred_heads)
accuracy_counts = comp_accu(cleaned_heading.split(), all_pred_heads, number_of_stop_words)

#show results
show_results(accuracy_counts,number_of_topics_pred, number_of_words_in_title, number_of_stop_words)



https://www.labnol.org/internet/buying-udemy-courses/31851//
<title>How to Save Money on Udemy Courses - No Discount Coupons Required</title>
How to Save Money on Udemy Courses - No Discount Coupons Required
how to save money on udemy courses no discount coupons required
how to save money on udemy courses no discount coupons required
11
save money udemy courses discount coupons required
7
Dictionary(292 unique tokens: ['tracker', 'overall', 'udemy', 'proceedsavoid', 'ad']...)
save save
udemy udemy
course course

Average Accuracy in 5 topics = 42.857142857142854 %
